# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane.

**Simulations**:
- **CTRL**: fixed surface concentration of hydrogen and methane
- **10H2**: as CTRL, but with surface H2 increased by 10%. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.

In [1]:
%%html
<style>
table {float:left}
</style>


| Models   | Simulations length | Average period |
|  :---    | :----              | :---           |
| OsloCTM3 | 20 years           |   year 20      |
| WACCM6   | 18 years           | last 5 years   |
| INCA     | 10 years           | last 3 years   |
| GFDL*    | 50 years           | last 40 years  | 





*GFDL simulations from Paulot et al. (2021) with H2 emissions (200 Tg/yr).*
Monthly mean data.

#### OUTLINE:
**PART I: Read model results**

1. Hydrogen budget
    1. H2 burden
    2. H2 loss
    3. H2 production
    4. Estimated H2 emissions
    5. H2 surface concentrations
    6. H2 lifetime
    7. H2 flux
2. Methane budget
    1. CH4 burden
    2. CH4 atmospheric loss
    3. CH4 surface concentrations
    4. CH4 lifetime
    5. CH4 flux
3. Ozone
    1. Change in troposheric ozone
    2. Change in stratosheric ozone
    3.  
    4. 
    5. Ozone radiative forcing
4. Stratospheric water wapor

**PART II: GWP calculations**


**PART III: Main results and tabels**

**Appendix with additional results**

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [3]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [4]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: REF.
agwp100_CO2 = 0.09170

#CH4 tau_strat[yr] Source: REF
tau_strat = 120.0

#CH4 tau_soil [yr] Source: REF
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44300

**Dry deposition adjustments** As there is large uncertanty in dry deposition, we set the dry deposition values in the control run to a high and a low value based on literature (80 and 50 Tg/yr). This will result in a range of GWP values. Adjust the perturbations with the same relative factors as in the control. 

In [5]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 80.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

# Part I: Read model results

In this part, model results are read from the input files.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [6]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,196,195,194,192,209
10H2,215,213,213,656,240
10CH4,197,195,195,nan,nan
deltaH2,18.7,18.7,18.6,464,30.8


### 1.2. H2 loss
Hydrogen loss happens through two main processes. The largest loss is through dry deposition at the ground. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 dry deposition [Tg/yr]

The models diagnose drydeposition based on their own schemes, but this is not used directly in the concentration driven runs, as the surface concentrations are fixed.

In [7]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,59.5,73.0,52.1,54.7,58.4
10H2,65.5,80.3,57.2,204,67.8
10CH4,59.5,73.0,52.1,nan,0.00


As drydeposition is uncertain, we can replace the models diagnosed drydeposition by a given value. In the concentration driven run, the drydeposition scheme do not impact the atmospheric composition. For emission driven runs, we also do adjust the drydeposition. Remark that we do not use the emission numbers from the emission driven runs directly, the fluxes is estimated based on burden and lifetimes.

In [8]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

#### H2 atmospheric loss [Tg/yr]

In [9]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,28.4,29.0,22.1,22.1,30.1
10H2,31.0,31.7,24.2,69.4,34.3
10CH4,27.6,28.2,21.6,nan,0.00


#### H2 total loss [Tg/yr]:

In [10]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,87.9,102,74.2,76.8,88.4
10H2,96.4,112,81.4,273,102
10CH4,87.2,101,73.7,nan,0.00


### 1.3. H2 production

#### H2 atm. prod [Tg/yr]

In [11]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,56.3,33.9,46.7,44.7,56.2
10H2,56.2,33.8,46.6,43.1,56.1
10CH4,59.2,36.4,48.9,nan,0.00


### 1.4. Estimated H2 emissions (Total loss = Total prod + emission)

In surface concentration driven runs, there are two unknowns; emissions and dry deposition. 
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. We adjust the dry deposition to take into account uncertainties in emissions and dry deposition on the GWP values.

In [12]:
df_h2_estemis = df_h2_atmloss.loc['CTRL'] + df_h2_drydep.loc['CTRL'] - df_h2_atmprod.loc['CTRL']
df_h2_estemis

OSLOCTM3       31.6
WACCM6-2deg    68.1
INCA           27.5
GFDL-emi       32.1
OSLOCTM3-emi   32.2
Name: CTRL, dtype: float64

### 1.5. H2 surface concentration [ppb]

In [13]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,532,532,532,534,567
10H2,585,585,585,"1,880",656
deltaH2,53.2,53.2,53.2,"1,346",88.7


### 1.6. H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

In [14]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,2.23,1.91,2.62,2.50,2.36
10H2,2.23,1.91,2.62,2.40,2.35
10CH4,2.26,1.93,2.64,nan,nan


#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided only by the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [15]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,6.91,6.70,8.79,8.66,6.95
10H2,6.94,6.73,8.82,9.44,6.98
10CH4,7.12,6.93,9.01,nan,nan


### 1.7. H2 flux (burden dived by lifetime) [Tg/yr]

The hydrogen flux is calculated as the burden divided by the hydrogen lifetime. The hydrogen lifetime is calculated by the burden divided by the total lifetime (drydep + atm.loss). (This is equal to the total loss, as it will be in steady state)

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks (add why). 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [16]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,87.9,102,74.2,76.8,88.4
10H2,96.4,112,81.4,273,102
10CH4,87.2,101,73.7,nan,nan
deltaH2,8.54,9.93,7.19,196,13.7
deltaCH4,-0.75,-0.86,-0.49,nan,nan


Description of the negative deltaCH4. In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

## 2. Methane results

### 2.1 CH4 burden [Tg]

In [17]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,"4,995","5,003","4,994","4,935","4,995"
10H2,"4,995","5,003","4,995","4,935","4,995"
10CH4,"5,496","5,505","5,494",nan,nan
deltaCH4,501,502,500,nan,nan


### 2.2 CH4 atmospheric loss [Tg/yr]

In [18]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,683,727,550,582,681
10H2,680,724,547,537,677
10CH4,728,772,588,nan,nan


### 2.3 CH4 surface concentration [ppb]

In [19]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,"1,813","1,808","1,813","1,804","1,813"
10CH4,"1,994","1,989","1,994",nan,nan
deltaCH4,181,181,181,nan,nan


### 2.4 CH4 lifetime [yr]

In [20]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,7.31,6.88,9.09,8.48,7.33
10H2,7.34,6.91,9.12,9.19,7.38
10CH4,7.55,7.13,9.35,nan,nan


#### Total CH4 lifetime [yr]

In [21]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,6.61,6.25,8.02,7.54,6.62
10H2,6.63,6.28,8.05,8.10,6.66
10CH4,6.80,6.46,8.23,nan,nan


### 2.5 CH4 flux (burden dived by total lifetime) [Tg/yr]

In [22]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
CTRL,756,800,622,654,754
10H2,753,797,620,609,750
10CH4,808,852,668,nan,nan
deltaH2,-2.67,-3.28,-2.10,-45.2,-4.38
deltaCH4,52.4,52.4,45.3,nan,nan


## 3. Ozone burden and RF

### 3.1 Change in tropospheric ozone (DU)

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. For GFDL-emi 10H2 is H2 perturbation and 10CH4 is H2+CH4 pert minus the H2 pert.

In [23]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.04,0.03,0.03,0.73,0.06
10CH4,0.83,0.88,0.54,0.76,nan


### 3.2 Change in stratospheric ozone (DU)

In [24]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,-0.03,0.00,-0.03,-0.23,0.02
10CH4,0.83,0.83,0.35,1.07,nan


### 3.3 Change in stratospheric ozone 60S-60N only (DU)

In [25]:
file = 'ozone_du_strat_6060.txt'
df_ozone_du_strat_6060 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_6060

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.00,0.00,-0.02,-0.22,nan
10CH4,0.62,0.66,0.28,0.68,nan


### 3.4 Change in stratospheric ozone 90S-60S and 60N-90N only (DU)

In [26]:
file = 'ozone_du_strat_gt60.txt'
df_ozone_du_strat_gt60 = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat_gt60

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,0.00,0.00,-0.01,-0.01,nan
10CH4,0.23,0.17,0.08,0.40,nan


### 3.5 Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone concentration. For GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4 pert minus the H2 pert.

In [27]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,1.86,1.73,1.12,34.0,3.08
10CH4,41.0,42.9,22.6,37.6,nan


In [28]:
df_ozone_rf['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_rf['OSLOCTM3'].loc['10CH4'] 
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
Scenario,,,,,
10H2,1.86,1.73,1.12,34.0,3.08
10CH4,41.0,42.9,22.6,37.6,41.0


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline. Preliminary results for OsloCTM3-emis.

In [29]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi
Scenario,,,,
10H2,1.48,0.51,0.53,1.33
10CH4,10.1,3.76,4.58,nan


In [30]:
df_h2o_rf['OSLOCTM3-emi'].loc['10CH4'] = df_h2o_rf['OSLOCTM3'].loc['10CH4'] 
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi
Scenario,,,,
10H2,1.48,0.51,0.53,1.33
10CH4,10.1,3.76,4.58,10.1


In [31]:
file = 'H2O_rf_gfdl.txt'
h2o_rf_gfdl = pd.read_csv(path + file, sep=',',index_col=0,header=0)
h2o_rf_gfdl

,GFDL
H2 [mW m-2],32.6
H2+CH4 [mW m-2],51.7



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [32]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3       6.23
WACCM6-2deg    5.36
INCA           7.39
GFDL-emi       6.86
OSLOCTM3-emi   6.48
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above, or move description here)... Why this includes feedback...

In [33]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

OSLOCTM3       0.31
WACCM6-2deg    0.33
INCA           0.29
GFDL-emi       0.23
OSLOCTM3-emi   0.32
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [34]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           4.00
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: surf_ch4_per_ch4_flux, dtype: float64

Set equal to OsloCTM3 concentrationdriven

In [35]:
df_surf_ch4_per_ch4_flux['OSLOCTM3-emi'] = df_surf_ch4_per_ch4_flux['OSLOCTM3']
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.46
WACCM6-2deg    3.45
INCA           4.00
GFDL-emi        nan
OSLOCTM3-emi   3.46
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [36]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv (REF Paolot)
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux

OSLOCTM3       1.08
WACCM6-2deg    1.14
INCA           1.17
GFDL-emi       1.00
OSLOCTM3-emi   1.11
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

Why we multiply by -1

In [37]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: h2_flux_per_ch4_flux, dtype: float64

In [38]:
df_h2_flux_per_ch4_flux['OSLOCTM3-emi'] = df_h2_flux_per_ch4_flux['OSLOCTM3']
df_h2_flux_per_ch4_flux

OSLOCTM3       0.01
WACCM6-2deg    0.02
INCA           0.01
GFDL-emi        nan
OSLOCTM3-emi   0.01
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

Explain why this is AGWP

In [39]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'





### Initialize H2 GWP table

In [40]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])

### Add methane GWP

In [41]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [42]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_h2o['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2+CH4 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'

#NBNB GFDL methane induced is included.

In [43]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'
df_h2o_rf_per_h2_flux['GFDL-emi'] = h2o_rf_gfdl['GFDL'].loc['H2 [mW m-2]']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux (Move this to the methane part?)
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [44]:
df_h2o_rf_per_ch4_flux['OSLOCTM3-emi'] = df_h2o_rf_per_ch4_flux['OSLOCTM3']
df_h2o_rf_per_ch4_flux

GFDL-emi        nan
INCA           0.10
OSLOCTM3       0.19
OSLOCTM3-emi   0.19
WACCM6-2deg    0.07
Name: h2o_rf_per_ch4_flux, dtype: float64

### Add stratospheric H2O GWP

In [45]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2

### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [46]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = (df_ozone_rf['GFDL-emi'].loc['10H2']+df_ozone_rf['GFDL-emi'].loc['10CH4'])/df_h2_flux['GFDL-emi'].loc['deltaH2']


In [47]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

In [48]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

In [49]:
df_ozone_rf_per_ch4_flux['OSLOCTM3-emi'] = df_ozone_rf_per_ch4_flux['OSLOCTM3']
df_ozone_rf_per_ch4_flux

OSLOCTM3       0.78
WACCM6-2deg    0.82
INCA           0.50
GFDL-emi        nan
OSLOCTM3-emi   0.78
Name: ozone_rf_per_ch4_flux, dtype: float64

### Add Ozone GWP

In [50]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2

### For the per flux table

In [51]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'


In [52]:
df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

In [53]:
df_strat_du_6060_ozone_per_ch4_flux = df_ozone_du_strat_6060.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_6060_ozone_per_ch4_flux.name = 'strat_du_6060_ozone_per_ch4_flux'

df_strat_du_6060_ozone_per_h2_flux = df_ozone_du_strat_6060.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_6060_ozone_per_h2_flux.name = 'strat_du_6060_ozone_per_h2_flux'

df_strat_du_gt60_ozone_per_ch4_flux = df_ozone_du_strat_gt60.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_gt60_ozone_per_ch4_flux.name = 'strat_du_gt60_ozone_per_ch4_flux'

df_strat_du_gt60_ozone_per_h2_flux = df_ozone_du_strat_gt60.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_gt60_ozone_per_h2_flux.name = 'strat_du_gt60_ozone_per_h2_flux'

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden, tropospheric concentration?

In [54]:
df_ch4_surfconc['OSLOCTM3-emi'].loc['deltaCH4']=df_ch4_surfconc['OSLOCTM3'].loc['deltaCH4']

In [55]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'


### Add methane induced O3 GWP

In [56]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.45
CH4,5.22,5.50,5.65,4.85,5.35
strat H2O,1.89,0.56,0.81,2.87,1.06
O3 CH4ind,2.67,2.94,1.59,nan,2.73
strat H2O CH4ind,nan,nan,nan,nan,nan


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [57]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'

### Add methane induced strat H2O GWP

In [58]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.45
CH4,5.22,5.50,5.65,4.85,5.35
strat H2O,1.89,0.56,0.81,2.87,1.06
O3 CH4ind,2.67,2.94,1.59,nan,2.73
strat H2O CH4ind,0.66,0.26,0.32,nan,0.67


# Methane GWP

Initialize CH4 GWP

In [59]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
#df_ch4_gwp

### Methane AGWP100 O3 [mW m-2 yr Tg-1]

Integrated O3 from CH4 RF [W m-2 yr Tg-1]

In [60]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'



### Add ozone GWP

In [61]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [62]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'

### Add methane GWP

In [63]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [64]:
print(df_h2o_rf.loc['10CH4'])
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'

OSLOCTM3       10.1
WACCM6-2deg    3.76
INCA           4.58
OSLOCTM3-emi   10.1
Name: 10CH4, dtype: float64


### Add Strat H2O GWP:

In [65]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Add GWP via H2

In [66]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()

# Part III: Main results and tables

## H2 GWP 100

In [67]:
#Save to file below. Where CH4 are split based on feedback factor:
#print(outputpath + 'table_h2_gwp.csv')
#df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')
df_h2_gwp.loc['total']=df_h2_gwp.sum()
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.45
CH4,5.22,5.50,5.65,4.85,5.35
strat H2O,1.89,0.56,0.81,2.87,1.06
O3 CH4ind,2.67,2.94,1.59,nan,2.73
strat H2O CH4ind,0.66,0.26,0.32,nan,0.67
total,12.8,11.2,10.1,11.7,12.3


## CH4 GWP 100

In [68]:
df_ch4_gwp.to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,8.55,8.92,5.43,nan,8.55
CH4,16.7,16.7,19.3,nan,16.7
strat H2O,2.10,0.78,1.10,nan,2.10
H2,0.18,0.18,0.11,nan,0.18
total,27.6,26.6,26.0,0.00,27.6


## Table per flux H2

In [69]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,df_ch4_flux_per_h2_flux,df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux,df_strat_du_ozone_per_h2_flux,
                            df_strat_du_6060_ozone_per_h2_flux,df_strat_du_gt60_ozone_per_h2_flux,
                            df_ozone_rf_per_h2_flux,df_h2o_rf_per_h2_flux],axis=1, sort=False)

#Save to file:
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_h2_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_h2_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,8.54,6.23,1.08,0.31,0.48,0.00,-0.00,0.00,0.00,0.22,0.17
WACCM6-2deg,9.93,5.36,1.14,0.33,0.50,0.00,0.00,0.00,0.00,0.17,0.05
INCA,7.19,7.39,1.17,0.29,0.52,0.00,-0.00,-0.00,-0.00,0.16,0.07
GFDL-emi,196,6.86,1.00,0.23,0.44,0.00,-0.00,-0.00,-0.00,0.17,0.17
OSLOCTM3-emi,13.7,6.48,1.11,0.32,0.49,0.00,0.00,nan,nan,0.22,0.10


## Table per flux CH4

In [70]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,df_trop_du_ozone_per_ch4_flux,df_strat_du_ozone_per_ch4_flux,
                            df_strat_du_6060_ozone_per_ch4_flux,df_strat_du_gt60_ozone_per_ch4_flux,
                            df_ozone_rf_per_ch4_flux,df_h2o_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [DU yr/Tg]',
               'strat_du_6060_ozone_per_ch4_flux':'Strat. ozone 60S-60N per flux [DU yr/Tg]',
               'strat_du_gt60_ozone_per_ch4_flux':'Strat. ozone >60S/N per flux [DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:
df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_ch4          

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],Trop. ozone per flux [DU yr/Tg],Strat. ozone per flux [DU yr/Tg],Strat. ozone 60S-60N per flux [DU yr/Tg],Strat. ozone >60S/N per flux [DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg]
OSLOCTM3,52.4,3.46,0.01,0.02,0.02,0.01,0.00,0.78,0.19
WACCM6-2deg,52.4,3.45,0.02,0.02,0.02,0.01,0.00,0.82,0.07
INCA,45.3,4.00,0.01,0.01,0.01,0.01,0.00,0.50,0.10
GFDL-emi,nan,nan,nan,nan,nan,nan,nan,nan,nan
OSLOCTM3-emi,nan,3.46,0.01,0.02,0.02,nan,nan,0.78,0.19


## H2 budget table

In [71]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],df_h2_lifetime.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],df_h2_drydep.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_estemis],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 lifetime [yrs]',
                        'H2 atm.loss[Tg/yr]',
                        'H2 drydep [Tg/yr]',
                        'H2 atm. prod [Tg/yr]',
                        'H2 atmospheric lifetime [yrs]',
                        'H2 estimated emissions [Tg/yr]']

df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2

,H2 burden [Tg],H2 lifetime [yrs],H2 atm.loss[Tg/yr],H2 drydep [Tg/yr],H2 atm. prod [Tg/yr],H2 atmospheric lifetime [yrs],H2 estimated emissions [Tg/yr]
OSLOCTM3,196,2.23,28.4,59.5,56.3,6.91,31.6
WACCM6-2deg,195,1.91,29.0,73.0,33.9,6.70,68.1
INCA,194,2.62,22.1,52.1,46.7,8.79,27.5
GFDL-emi,192,2.50,22.1,54.7,44.7,8.66,32.1
OSLOCTM3-emi,209,2.36,30.1,58.4,56.2,6.95,32.2


## CH4 budget table

In [72]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],df_ch4_lifetime.loc['CTRL'],
                           df_ch4_loss.loc['CTRL']],axis=1)
df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 lifetime [yrs]','CH4 atm.loss[Tg/yr]'] 
df_budget_ch4

,CH4 burden [Tg],CH4 lifetime [yrs],CH4 atm.loss[Tg/yr]
OSLOCTM3,"4,995",7.31,683
WACCM6-2deg,"5,003",6.88,727
INCA,"4,994",9.09,550
GFDL-emi,"4,935",8.48,582
OSLOCTM3-emi,"4,995",7.33,681


# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [73]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

OSLOCTM3       2.76
WACCM6-2deg    2.78
INCA           2.76
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: ch4_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

Add explanation... 

In [74]:
df_w_o_feedback =df_ch4_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback

OSLOCTM3       2.65
WACCM6-2deg    2.48
INCA           3.30
GFDL-emi        nan
OSLOCTM3-emi    nan
dtype: float64

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [75]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4

OSLOCTM3       1.31
WACCM6-2deg    1.39
INCA           1.21
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: feedback_factor_ch4, dtype: float64

Split the CH4 GWP into direkt and indirect based on the feedback factor. For the GWP figure

In [76]:
feedback_factor = df_feedback_factor_ch4
feedback_frac = 1.0 - (1.0/feedback_factor)
feedback_frac.name = 'feedback_frac'
feedback_frac

OSLOCTM3       0.24
WACCM6-2deg    0.28
INCA           0.18
GFDL-emi        nan
OSLOCTM3-emi    nan
Name: feedback_frac, dtype: float64

In [77]:
#Save to file:

df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-feedback_frac)
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*feedback_frac

df_h2_gwp['GFDL-emi'].loc['CH4dir'] = df_h2_gwp['GFDL-emi'].loc['CH4']
df_h2_gwp['OSLOCTM3-emi'].loc['CH4dir'] = df_h2_gwp['OSLOCTM3-emi'].loc['CH4']
df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi
O3,2.38,1.90,1.69,3.98,2.45
strat H2O,1.89,0.56,0.81,2.87,1.06
O3 CH4ind,2.67,2.94,1.59,nan,2.73
strat H2O CH4ind,0.66,0.26,0.32,nan,0.67
CH4dir,3.99,3.95,4.66,4.85,5.35
CH4indir,1.23,1.55,0.99,nan,nan


## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [78]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

OSLOCTM3       0.35
WACCM6-2deg    0.35
INCA           0.35
GFDL-emi       0.34
OSLOCTM3-emi   0.35
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [79]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

OSLOCTM3       6.36
WACCM6-2deg    5.44
INCA           7.50
GFDL-emi       7.25
OSLOCTM3-emi   6.79
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

In [80]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

OSLOCTM3       0.98
WACCM6-2deg    0.99
INCA           0.99
GFDL-emi       0.95
OSLOCTM3-emi   0.95
Name: feedback_factor_h2, dtype: float64